<a href="https://colab.research.google.com/github/nicolaiberk/FrameCompetition/blob/main/Doc2Vec_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/MyDrive/frameComp")

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import string
import csv
from datetime import datetime
import re
import smart_open
import multiprocessing

import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer

PunctTokenizer = RegexpTokenizer(r'\w+')

import sys
csv.field_size_limit(sys.maxsize)

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
if not os.path.isfile('news_merged.csv'):
  # get annotated migration articles
  !wget -O news_merged.csv https://www.dropbox.com/s/rij19gzmka99a1m/news_merged.csv?dl=0

# Time for the Doc2 Vec model
See tutorial [here](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#define-a-function-to-read-and-preprocess-text).

In [ ]:
## alternative approach using iterator from https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html 
class MyCorpus:
    def __iter__(self):
        with open("news_merged.csv", encoding = "UTF-8") as f:
          reader = csv.reader(f)

          for i, line in tqdm(enumerate(reader)):

            if i == 0:
              paper_pos = np.argmax([l == 'paper' for l in line])
              text_pos = np.argmax([l == 'text' for l in line])
              date_pos = np.argmax([l == 'date_clean' for l in line])


            # preprocess text
            text = PunctTokenizer.tokenize(line[text_pos])
            
            # define tag
            tag = line[paper_pos]+line[date_pos][0:7]

            # return data in appropriate format
            yield gensim.models.doc2vec.TaggedDocument(text, [tag])

In [ ]:
print(f"Started setting up the model at {datetime.now()}")
d2v_model = gensim.models.doc2vec.Doc2Vec(MyCorpus(),
                                          vector_size=300, 
                                          min_count=100, 
                                          epochs=5, 
                                          workers = multiprocessing.cpu_count(), 
                                          docvecs_mapfile = "docvecs_mapfile")
print(f"Finished training at {datetime.now()}")

Started setting up the model at 2022-12-06 17:20:23.780952


4932469it [28:52, 2847.63it/s]
4932469it [1:34:12, 872.59it/s]
4932469it [1:37:00, 847.48it/s]
4932469it [1:44:40, 785.35it/s]
4932469it [1:47:48, 762.60it/s]
4932469it [1:38:54, 831.18it/s]


Finished training at 2022-12-07 02:13:00.394215


In [ ]:
d2v_model.save("np_doc_emb")
print("model saved")

model saved
